In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_ac.txt
/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_ae.txt
/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_af.txt
/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_ad.txt
/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_aa.txt
/kaggle/input/0-25a-split/0.25a/NoNan0.25aa_part_ab.txt
/kaggle/input/finaltestmultiscale-r-3/finalTestMultiScale r_3- Cloud.txt
/kaggle/input/model-a/model_a.pkl
/kaggle/input/0-25ab/0.25aa.txt
/kaggle/input/testmulti/finalTestMultiScale.txt
/kaggle/input/multiscale-r-3/multiScale_r_3 - Cloud.txt
/kaggle/input/model-ad/model_ad.pkl
/kaggle/input/model-ac/model_ac.pkl
/kaggle/input/model-ab/model_ab.pkl


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [3]:
import numpy as np
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

def load_data(filename):
    # Read the first line to get the column names and set desired feature order
    with open(filename, 'r') as file:
        column_names = file.readline().strip().split()
    column_names[0] = 'X'

    feature_order = [
        'X', 'Y', 'Z', 'R', 'G', 'B',
        'Planarity_(0.1)', 'Linearity_(0.1)', 'PCA1_(0.1)', 'Sphericity_(0.1)', 'Verticality_(0.1)', '1st_eigenvalue_(0.1)',
        'Planarity_(0.125)', 'Linearity_(0.125)', 'PCA1_(0.125)', 'Sphericity_(0.125)', 'Verticality_(0.125)', '1st_eigenvalue_(0.125)',
        'Sphericity_(0.25)', 'Verticality_(0.25)', '1st_eigenvalue_(0.25)',
        'Planarity_(0.5)', 'Linearity_(0.5)', 'PCA1_(0.5)', 'Sphericity_(0.5)', 'Verticality_(0.5)', '1st_eigenvalue_(0.5)',
        'Planarity_(1)', 'Linearity_(1)', 'PCA1_(1)', 'Sphericity_(1)', 'Verticality_(1)', '1st_eigenvalue_(1)',
        '3rd_eigenvalue_(2)', 'Linearity_(2)', 'PCA1_(2)', 'Sphericity_(2)', 'Verticality_(2)', '1st_eigenvalue_(2)', '2nd_eigenvalue_(2)',
        'Planarity_(3)', 'Linearity_(3)', 'PCA1_(3)', 'Sphericity_(3)', 'Verticality_(3)', '1st_eigenvalue_(3)', '2nd_eigenvalue_(3)', '3rd_eigenvalue_(3)'
    ]

    target_points_per_class = 200000
    # Read the data skipping the first line
    data = np.genfromtxt(filename, skip_header=1)

    # Remove rows with NaN values
    data = data[~np.isnan(data).any(axis=1)]

    # Reorder the features based on the desired order
    feature_indices = [column_names.index(feature) for feature in feature_order[:]]  # Exclude 'Classification'
    features = data[:, feature_indices]

    # Extract labels
    label_index = column_names.index('Classification')
    labels = data[:, label_index].astype(int)

    # Perform class balancing
    class_counts = Counter(labels)
    balanced_features = []
    balanced_labels = []
    for cls, count in class_counts.items():
        cls_features = features[labels == cls]
        cls_labels = np.full((count,), cls, dtype=int)
        if count < target_points_per_class:
            # Upsample minority class
            cls_features_resampled = resample(cls_features, n_samples=target_points_per_class, replace=True, random_state=42)
            cls_labels_resampled = resample(cls_labels, n_samples=target_points_per_class, replace=True, random_state=42)
        else:
            # Downsample majority class
            cls_features_resampled = resample(cls_features, n_samples=target_points_per_class, replace=False, random_state=42)
            cls_labels_resampled = resample(cls_labels, n_samples=target_points_per_class, replace=False, random_state=42)
        balanced_features.append(cls_features_resampled)
        balanced_labels.append(cls_labels_resampled)

    balanced_features = np.concatenate(balanced_features)
    balanced_labels = np.concatenate(balanced_labels)

    # Print class counts
    print("Class Counts:")
    for cls, count in sorted(class_counts.items()):
        print(f"Class {cls}: {count}")

    # Print balanced label counts
    balanced_label_counts = Counter(balanced_labels)
    print("\nBalanced Label Counts:")
    for cls, count in sorted(balanced_label_counts.items()):
        print(f"Class {cls}: {count}")

    return balanced_features, balanced_labels


filename = '/kaggle/input/multiscale-r-3/multiScale_r_3 - Cloud.txt' 
points, labels = load_data(filename)
num_classes = len(np.unique(labels))

print("Total number of classes:", num_classes)


Class Counts:
Class 0: 1629365
Class 1: 224671
Class 2: 45141
Class 3: 112805
Class 4: 642025
Class 5: 147937
Class 6: 46888
Class 7: 438052
Class 9: 18636
Class 10: 2861

Balanced Label Counts:
Class 0: 200000
Class 1: 200000
Class 2: 200000
Class 3: 200000
Class 4: 200000
Class 5: 200000
Class 6: 200000
Class 7: 200000
Class 9: 200000
Class 10: 200000
Total number of classes: 10


In [4]:
# print(points[11])

In [5]:
nan_rows = np.any(np.isnan(points), axis=1)
num_nan_rows = np.sum(nan_rows)

print(f"Number of rows containing NaN values: {num_nan_rows}")
total_rows = points.shape[0]
print(f"Total number of rows: {total_rows}")
print(f"Total number of labels",labels.shape)

Number of rows containing NaN values: 0
Total number of rows: 2000000
Total number of labels (2000000,)


In [7]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_val, y_train, y_val = train_test_split(points, labels, test_size=0.2, random_state=42)

# Convert data into DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set parameters for XGBoost
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': 11,       # Number of classes
    'eval_metric': 'merror'         # Evaluation metric: Multiclass classification error rate
}

# Train the model with early stopping
num_rounds = 1000  # Maximum number of boosting rounds
early_stopping_rounds = 20  # Stop if no improvement in 20 rounds
watchlist = [(dtrain, 'train'), (dval, 'validation')]  # Data to monitor for early stopping

model = xgb.train(params, dtrain, num_rounds, evals=watchlist, early_stopping_rounds=early_stopping_rounds)

# Predictions on validation set
y_pred = model.predict(dval)

# Calculate accuracy on validation set
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


[0]	train-merror:0.11701	validation-merror:0.11739
[1]	train-merror:0.09420	validation-merror:0.09474
[2]	train-merror:0.08695	validation-merror:0.08720
[3]	train-merror:0.08190	validation-merror:0.08193
[4]	train-merror:0.07662	validation-merror:0.07686
[5]	train-merror:0.07238	validation-merror:0.07258
[6]	train-merror:0.06890	validation-merror:0.06918
[7]	train-merror:0.06408	validation-merror:0.06439
[8]	train-merror:0.06022	validation-merror:0.06051
[9]	train-merror:0.05683	validation-merror:0.05695
[10]	train-merror:0.05264	validation-merror:0.05299
[11]	train-merror:0.05025	validation-merror:0.05058
[12]	train-merror:0.04768	validation-merror:0.04800
[13]	train-merror:0.04460	validation-merror:0.04507
[14]	train-merror:0.04208	validation-merror:0.04251
[15]	train-merror:0.04004	validation-merror:0.04045
[16]	train-merror:0.03774	validation-merror:0.03825
[17]	train-merror:0.03600	validation-merror:0.03665
[18]	train-merror:0.03429	validation-merror:0.03492
[19]	train-merror:0.03

In [ ]:
# from joblib import load

# # Load the model from the .pkl file
# random_forest = load('/kaggle/input/model-ad/model_ad.pkl')
# random_forest.fit(points,labels)

In [ ]:
# # Predict on the test set
# y_pred = random_forest.predict(X_test)
# y_pred


In [ ]:
# # Predict on the test set
# y_pred = random_forest.predict(points)
# y_pred


In [ ]:
# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [ ]:
# # Calculate accuracy
# accuracy = accuracy_score(labels, y_pred)
# print("Accuracy:", accuracy)

In [9]:
import joblib
joblib.dump(model, 'r_3_xgboost.pkl')


['r_3_xgboost.pkl']

In [10]:
import numpy as np
from collections import Counter
from sklearn.utils import resample

def load_testData(filename):
     # Read the first line to get the column names and set desired feature order
    with open(filename, 'r') as file:
        column_names = file.readline().strip().split()
    column_names[0] = 'X'

    feature_order = [
    'X', 'Y', 'Z', 'R', 'G', 'B', 
    'Planarity_(0.1)', 'Linearity_(0.1)', 'PCA1_(0.1)', 'Sphericity_(0.1)', 'Verticality_(0.1)', '1st_eigenvalue_(0.1)',
    'Planarity_(0.125)', 'Linearity_(0.125)', 'PCA1_(0.125)', 'Sphericity_(0.125)', 'Verticality_(0.125)', '1st_eigenvalue_(0.125)',
    'Sphericity_(0.25)', 'Verticality_(0.25)', '1st_eigenvalue_(0.25)',
    'Planarity_(0.5)', 'Linearity_(0.5)', 'PCA1_(0.5)', 'Sphericity_(0.5)', 'Verticality_(0.5)', '1st_eigenvalue_(0.5)',
    'Planarity_(1)', 'Linearity_(1)', 'PCA1_(1)', 'Sphericity_(1)', 'Verticality_(1)', '1st_eigenvalue_(1)',
    '3rd_eigenvalue_(2)', 'Linearity_(2)', 'PCA1_(2)', 'Sphericity_(2)', 'Verticality_(2)', '1st_eigenvalue_(2)', '2nd_eigenvalue_(2)',
    'Planarity_(3)', 'Linearity_(3)', 'PCA1_(3)', 'Sphericity_(3)', 'Verticality_(3)', '1st_eigenvalue_(3)', '2nd_eigenvalue_(3)', '3rd_eigenvalue_(3)'
    ]

    max_points_per_class = 200000
    # Read the data skipping the first line
    data = np.genfromtxt(filename, skip_header=1)
    
    # Remove rows with NaN values
    data = data[~np.isnan(data).any(axis=1)]
    
    # Reorder the features based on the desired order
    feature_indices = [column_names.index(feature) for feature in feature_order[:]]  # Exclude 'Classification'
    features = data[:, feature_indices]
    
    # Extract labels
    label_index = column_names.index('Classification')
    labels = data[:, label_index].astype(int)
    
    return features,labels
 

# filename = '/kaggle/input/multiscale/multiScale.txt' 
# points, labels = load_data(filename)
# num_classes = len(np.unique(labels))

# print("Total number of classes:", num_classes)


In [11]:
points,labels = load_testData('/kaggle/input/finaltestmultiscale-r-3/finalTestMultiScale r_3- Cloud.txt')

In [12]:
dtest = xgb.DMatrix(points)

# Make predictions on the test data
y_pred = model.predict(dtest)

In [13]:
# # Predict on the test set
# y_pred = random_forest.predict(points)
# y_pred


In [14]:
# Calculate accuracy
accuracy = accuracy_score(labels, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7223661758200088


In [15]:
from sklearn.metrics import f1_score

f1 = f1_score(labels, y_pred, average='weighted')  
print("F1-score:", f1)

F1-score: 0.7056809492724536


In [16]:
def save_data(points, predicted_labels, filename):
    with open(filename, 'w') as file:
        # Write comment line with column names
        file.write("// X Y Z R G B predictedClassification\n")
        # Write data lines
        for i in range(len(points)):
            point = points[i][:6]
            predicted_label = predicted_labels[i]
            data_str = ' '.join([str(coord) for coord in point]) + f' {predicted_label}\n'
            file.write(data_str)

In [17]:
save_data(points, y_pred, "r_3_Final_model_result_xgboost.txt")
